In [17]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://www.lianjia.com"
html = urlopen(url).read()
bsobj = BeautifulSoup(html, "html5lib")

city_tags = bsobj.find("div", {"class":"fc-main clear"}).findChildren("a")

with open("./citys.csv", "w") as f:
    writ = csv.writer(f)
    for city_tag in city_tags:
        city_url = city_tag.get("href")
        city_name = city_tag.get_text()
        writ.writerow((city_name, city_url))
        print (city_name, city_url)

北京 https://bj.lianjia.com/
保亭 http://you.lianjia.com/bt
保定 https://bd.fang.lianjia.com/
成都 https://cd.lianjia.com/
重庆 https://cq.lianjia.com/
长沙 https://cs.lianjia.com/
澄迈 http://you.lianjia.com/cm
承德 https://chengde.fang.lianjia.com/
滁州 https://cz.fang.lianjia.com/
大连 https://dl.lianjia.com/
东莞 https://dg.lianjia.com/
儋州 http://you.lianjia.com/dz
定安 http://you.lianjia.com/da
大理 http://you.lianjia.com/dl1
德阳 https://dy.fang.lianjia.com/
佛山 https://fs.lianjia.com/
广州 https://gz.lianjia.com/
杭州 https://hz.lianjia.com/
惠州 https://hui.lianjia.com/
海口 https://hk.lianjia.com/
合肥 https://hf.lianjia.com/
衡水 https://hs.fang.lianjia.com/
黄冈 https://hg.fang.lianjia.com/
邯郸 https://hd.fang.lianjia.com/
济南 https://jn.lianjia.com/
嘉兴 https://jx.fang.lianjia.com/
晋中 https://jz.fang.lianjia.com/
昆明 https://km.fang.lianjia.com/
陵水 http://you.lianjia.com/ls
廊坊 https://lf.lianjia.com/
临高 http://you.lianjia.com/lg
乐东 http://you.lianjia.com/ld
龙岩 https://ly.fang.lianjia.com/
乐山 https://leshan.fang.lianjia.

In [ ]:
import sys
import csv
def get_city_dict():
    city_dict = {}
    with open("citys.csv", "r") as f:
        reader = csv.reader(f)
        for city in reader:
            print(city)
            city_dict[city[0]] = city[1]
    return city_dict

city_dict = get_city_dict()
for city_name in city_dict.keys():
    print (city_name)
    
input_city_name = input("Please input city name:")
city_url = city_dict.get(input_city_name)
if city_url:
    print(input_city_name, city_url)
else:
    print("input error")


In [19]:
def get_district_dict(url):
    district_dict = {}
    html = urlopen(url).read()
    bsobj = BeautifulSoup(html, "html5lib")
    roles = bsobj.find("div", {"data-role":"ershoufang"}).findChildren("a")
    for role in roles:
        district_url = role.get("href")
        district_name = role.get_text()
        district_dict[district_name] = district_url
    return district_dict
def run():
    city_dict = get_city_dict()
    for city in city_dict.keys():
        print(city)
    input_city = input("Please input city name:")
    city_url = city_dict.get(input_city)
    if not city_url:
        print("input error")
        sys.exit()
    ershoufang_city_url = city_url + "ershoufang"
    district_dict = get_district_dict(ershoufang_city_url)
    for district in district_dict.keys():
        print(district)
    input_district = input("Please input area:")
    district_url = district_dict.get(input_district)
    if not district_url:
        print("input error")
        sys.exit()
    house_info_url = city_url + district_url[1:]
    #print(house_info_url)
    house(house_info_url)
if __name__ == "__main__":
    run()

FileNotFoundError: [Errno 2] No such file or directory: 'citys.csv'

In [19]:
import sys
import re
import csv
import urllib

def get_bsobj(url):
    page = urllib.request.urlopen(url)
    if page.getcode() == 200:
        html = page.read()
        bsobj = BeautifulSoup(html, "html5lib")
        return bsobj
    else:
        print("page error")
        sys.exit()
def get_house_info_list(url):
    house_info_list = []
    bsobj = get_bsobj(url)
    if not bsobj:
        return None
    house_list = bsobj.find_all("li", {"class":"clear"})
    for house in house_list:
        title = house.find("div", {"class":"title"}).get_text()
        info = house.find("div", {"class":"houseInfo"}).get_text().split("|")
        block = info[0].strip()
        house_type = info[1].strip()
        size_info = info[2].strip()
        size = re.findall(r"\d+", size_info)[0]
        price_info = house.find("div", {"class":"totalPrice"}).span.get_text()
        price = re.findall(r"\d+", price_info)[0]
        
        house_info_list.append({
            "title":title,
            "price":price,
            "size":int(size),
            "block":block,
            "house_type":house_type
        })
    return house_info_list
def house(url):
    house_info_list = []
    for i in range(3):
        new_url = url + 'pg' + str(i+1)
        house_info_list.extend(get_house_info_list(new_url))
        if house_info_list:
            with open("./house/csv", "wb+") as f:
                writer = csv.writer(f, delimiter = "|")
                for house_info in house_info_list:
                    title = house_info.get("title")
                    price = house_info.get("price")
                    size = house_info.get("size")
                    block = house_info.get("block")
                    house_type = house_info.get("house_type")
                    writer.writerow([str(title), int(price), int(size), str(block), str(house_type)])

In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt
price, size = numpy.loadtxt('house.csv', delimiter='|', usecols=(1,2), unpack=True)
print(price)
print(size)
price_mean = np.mean(price)
szie_mean = np.mean(size)
print("平均价格:", price_mean)
print("平均面积:", size_mean)
price_var = np.var(price)
size_var = np.var(size)
print("价格的方差：", price_var)
print("面积的方差：", size_var)

In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
price, size = np.loadtxt('house.txt', delimiter='|', usecols=(1,2), unpack=True)
plt.figure()
plt.subplot(211)
plt.title("/ 10000RMB")
plt.hist(price, bins=20)
plt.subplot(212)
plt.xlable("/ m**2")
plt.hist(size, bins=20)
plt.figure(2)
plt.title("price")
plt.plot(price)
plt.show()


OSError: house.txt not found.